In [1]:
import tensorflow as tf
#tf.compat.v1.enable_eager_execution()
import numpy as np
from numpy import unique
import cv2
import os
from tensorflow.python.client import device_lib
from sklearn.model_selection import train_test_split
from IPython.display import clear_output
import matplotlib.pyplot as plt
import gc
import dask
import dask.array as da
import dask.dataframe as dd
import random
import xarray as xr
from dask.array.image import imread
#os.environ['KMP_DUPLICATE_LIB_OK']='True'
print("done")
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)
gpus = tf.config.experimental.list_physical_devices('GPU')
#tf.config.experimental.set_virtual_device_configuration(
    #gpus[0],
   # [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)])
face_cascade = cv2.CascadeClassifier('faces.xml')

done
1 Physical GPUs, 1 Logical GPUs


In [2]:
print("START")
def imread_f(filename):
    item=cv2.imread(filename)
    return item
def preprocessing(item):
    gray = cv2.cvtColor(item, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x, y, w, h) in faces:
        new_im=item[y:y+h,x:x+w,:]
        new_im=cv2.resize(new_im, (160, 160)).astype(np.float32)
        new_im=(new_im-np.mean(new_im))/np.std(new_im)
        return new_im
    res=cv2.resize(item, (160, 160)).astype(np.float32)
    res=(res-np.mean(res))/np.std(res)
    return res

START


In [3]:
def face_detector(path):
    im=cv2.imread(path)
    gray=cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    faces=face_cascade.detectMultiScale(gray, 1.3, 5)
    photo=None
    c=0
    for (x, y, w, h) in faces:
        photo=im[y:y+h,x:x+w,:]
        photo=cv2.resize(photo, (160, 160))
        im_=photo.astype(np.float32)
        im_=(im_-np.mean(im_))/np.std(im_)
        c=1
    if c==1:
        return im_.reshape(1, 160, 160, 3)
    return None
def if_n(img, l):
    if img is None:
        return
    else:
        l.append(img)
        return
l_positive=[]
gpu_available = tf.test.is_gpu_available()
print(gpu_available)
path_ ="./Humans"
pathes=os.listdir("./Humans")
images_negative=[]
im_1, anchor = face_detector('./positive_anchor/photo_positive.jpg'), face_detector('./positive_anchor/photo_anchor.jpg')
if_n(im_1, l_positive)
if_n(face_detector('./positive_anchor/p_2.jpg'), l_positive)
if_n(face_detector('./positive_anchor/p_3.jpg'), l_positive)
if_n(face_detector('./positive_anchor/p_4.jpg'), l_positive)
if_n(face_detector('./positive_anchor/p_5.jpg'), l_positive)
l_positive=np.array(l_positive)
print("END")
print(l_positive[0])
print("END")
print(np.mean(l_positive[0]))
print(np.mean(anchor))
l_p=len(l_positive)
c=4


ims=imread('./Humans/img_align_celeba/*.jpg', imread=imread_f, preprocess=preprocessing).astype(np.float32)
ims.shape
#ims=ims.to_dask_array(lengths=True)
#ims=ims.reshape((ims.shape[0], 160, 160, 3))
#ims=ims.map_blocks(func, (x), dtype=np.float32)
#ims=ims.map_overlap(func, x)


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
True
END
[[[[-0.0545023   0.08628052  0.06868267]
   [-0.0545023   0.08628052  0.08628052]
   [-0.01930659  0.13907409  0.13907409]
   ...
   [ 0.33265045  0.42063972  0.40304187]
   [ 0.40304187  0.49103114  0.4734333 ]
   [ 0.36784616  0.45583543  0.43823758]]

  [[-0.00170874  0.08628052  0.08628052]
   [-0.00170874  0.08628052  0.10387838]
   [-0.00170874  0.08628052  0.13907409]
   ...
   [ 0.3502483   0.45583543  0.43823758]
   [ 0.3502483   0.45583543  0.43823758]
   [ 0.40304187  0.49103114  0.4734333 ]]

  [[-0.00170874  0.06868267  0.13907409]
   [-0.00170874  0.05108482  0.12147623]
   [-0.03690444  0.01588911  0.10387838]
   ...
   [ 0.36784616  0.45583543  0.43823758]
   [ 0.40304187  0.50862896  0.49103114]
   [ 0.38544402  0.45583543  0.45583543]]

  ...

  [[-1.6559069  -1.1279713  -1.6559069 ]
   [-1.6031133  -1.1807649  -1.5855155 ]
   [-1.5503198  -1.2511563  -1.4975262 ]
   ...
   [-1.4

(202599, 160, 160, 3)

In [4]:
ims[1].compute()

array([[[-1.8733778 , -1.8733778 , -1.8733778 ],
        [-1.8733778 , -1.8908063 , -1.8908063 ],
        [-1.8559494 , -1.8908063 , -1.8908063 ],
        ...,
        [-1.0890969 , -0.86252683,  0.07861026],
        [-1.2110962 , -1.0019546 , -0.06081746],
        [-1.3330954 , -1.1239538 , -0.18281671]],

       [[-1.8733778 , -1.8908063 , -1.8908063 ],
        [-1.8559494 , -1.8908063 , -1.8908063 ],
        [-1.8559494 , -1.8908063 , -1.8908063 ],
        ...,
        [-1.1588107 , -0.96709764,  0.0088964 ],
        [-1.2110962 , -1.0019546 , -0.043389  ],
        [-1.2459531 , -1.05424   , -0.07824592]],

       [[-1.8385209 , -1.8733778 , -1.9082347 ],
        [-1.8385209 , -1.8733778 , -1.8908063 ],
        [-1.8385209 , -1.8908063 , -1.8908063 ],
        ...,
        [-1.2285246 , -1.0368115 , -0.02596053],
        [-1.1936677 , -1.0019546 ,  0.0088964 ],
        [-1.1588107 , -0.96709764,  0.06118179]],

       ...,

       [[-0.23510212, -0.3919583 , -0.7230991 ],
        [-0

In [61]:
def convert(image, model):
  c_im=model.predict_on_batch(image.reshape(1, 160, 160, 3))
  return c_im
class CustomLoss(tf.keras.losses.Loss):
  def __init__(self, anchor, positive, y_pred):
    super().__init__()
    self.anchor=anchor
    self.positive=positive
    self.y_pred=y_pred
  def call(self, y_pred):
    y_pred_c=self.y_pred
    anchor, positive = self.anchor, self.positive
    print(y_pred.shape)
    pos_d=tf.math.sqrt(tf.math.reduce_sum(tf.square(tf.subtract(anchor, positive)), axis=1))
    neg_d=tf.math.sqrt(tf.math.reduce_sum(tf.square(tf.subtract(anchor, y_pred_c)), axis=1))
    #res=tf.add(tf.subtract(neg_d, pos_d), 0.2)
    res=tf.subtract(pos_d, neg_d)
    res=tf.add(res, 0.3)
    res=tf.boolean_mask(res, res>=0.2)
    fin=tf.math.reduce_sum(res, axis=0)
    return fin
loss_tracker = tf.keras.metrics.Mean(name="loss")
class CustomModel(tf.keras.Model):
  def __init__(self, anchor, positive):
    super(CustomModel, self).__init__()
    self.anchor=anchor
    self.positive=positive
    self.d=tf.keras.layers.Dropout(0.8)
    self.l_inception = self.get_inception_two()
    self.l_1=tf.keras.layers.Conv2D(input_shape=(160, 160, 3),
                            kernel_size=(6,6),
                            strides=(2,2),
                            filters=64,
                            activation=tf.keras.layers.LeakyReLU(alpha=0.3))
    self.l_2_1=tf.keras.layers.BatchNormalization()
    self.l_2=tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2))
    self.l_4_1=tf.keras.layers.BatchNormalization()
    self.l_4=tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2))
    self.l_7_1=tf.keras.layers.BatchNormalization()
    self.l_10_1=tf.keras.layers.BatchNormalization()
    self.l_10=tf.keras.layers.AveragePooling2D(pool_size=(3,3))
    self.l_13=tf.keras.layers.Flatten()
    self.l_13_1=tf.keras.layers.Lambda(lambda  x: tf.math.l2_normalize(x))
    self.l_14=tf.keras.layers.Dense(128, activation="sigmoid")
    self.l_15=tf.keras.layers.Lambda(lambda  x: tf.math.l2_normalize(x, 1, 1e-10))
  def get_inception(self):
    l_incept=[]
    n=1
    for x in range(4):
        l=[]
        l_4=tf.keras.layers.Conv2D(kernel_size=(1,1), filters=64*n, activation="tanh", strides=(1,1))
        l_5=tf.keras.layers.Conv2D(kernel_size=(1,1), filters=64*n , activation="tanh", strides=(1,1))
        l_6=tf.keras.layers.Conv2D(kernel_size=(1,1), filters=32*n, activation="tanh", strides=(1,1))
        l_7_f=tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2))
        l_9_f=tf.keras.layers.Conv2D(kernel_size=(3,3), padding="same", strides=(1,1), filters=8*n, activation="tanh")
        l_9_f_1=tf.keras.layers.Conv2D(kernel_size=(2,2), strides=(2,2), filters=64*n, activation="tanh")
        l_10_f=tf.keras.layers.Conv2D(kernel_size=(5,5), padding="same", strides=(1,1), filters=8*n, activation="tanh")
        l_10_f_1=tf.keras.layers.Conv2D(kernel_size=(2,2), strides=(2,2), filters=128*n, activation="tanh")
        l.append(l_4)
        l.append(l_5)
        l.append(l_6)
        l.append(l_7_f)
        l.append(l_9_f)
        l.append(l_9_f_1)
        l.append(l_10_f)
        l.append(l_10_f_1)
        l_incept.append(l)
        if n==2:
            n=n+1
        if n==4:
            n=n+1
        if n==6:
            n=n+2
        n=n+1
    return l_incept
  def get_inception_two(self):
    l_incept=[]
    n=1
    z=0
    c=0
    k=2
    for x in range(8):
        z=z+1
        l=[]
        if z%2==0:
            l_4=tf.keras.layers.Conv2D(kernel_size=(1,1), kernel_initializer="he_normal", filters=32*n, activation=tf.keras.layers.LeakyReLU(alpha=0.3), strides=(1,1))
            l_5=tf.keras.layers.Conv2D(kernel_size=(1,1), kernel_initializer="he_normal", filters=32*n, activation=tf.keras.layers.LeakyReLU(alpha=0.3),strides=(1,1))
            l_6=tf.keras.layers.Conv2D(kernel_size=(1,1), kernel_initializer="he_normal", filters=32*n, activation=tf.keras.layers.LeakyReLU(alpha=0.3), strides=(1,1))
            l_7_f=tf.keras.layers.MaxPooling2D(pool_size=(3,3), padding="same", strides=(1,1))
            l_9_f=tf.keras.layers.Conv2D(kernel_size=(3,3), kernel_initializer="he_normal", padding="same", strides=(1,1), filters=92*n, activation=tf.keras.layers.LeakyReLU(alpha=0.3))
            l_10_f=tf.keras.layers.Conv2D(kernel_size=(5,5), kernel_initializer="he_normal", padding="same", strides=(1,1), filters=64*n, activation=tf.keras.layers.LeakyReLU(alpha=0.3))
            l_11_f=tf.keras.layers.Conv2D(kernel_size=(1,1), activation=tf.keras.layers.LeakyReLU(alpha=0.3), kernel_initializer="he_normal", strides=(1,1), filters=32*n)
        else:
            if z==7:
                l_4=tf.keras.layers.Conv2D(kernel_size=(1,1), filters=32*n, kernel_initializer="he_normal", activation=tf.keras.layers.LeakyReLU(alpha=0.3), strides=(1,1))
                l_5=tf.keras.layers.Conv2D(kernel_size=(1,1), filters=32*n, kernel_initializer="he_normal", activation=tf.keras.layers.LeakyReLU(alpha=0.3),strides=(1,1))
                l_6=tf.keras.layers.Conv2D(kernel_size=(1,1), filters=32*n, activation=tf.keras.layers.LeakyReLU(alpha=0.3), strides=(1,1))
                l_7_f=tf.keras.layers.MaxPooling2D(pool_size=(3,3), padding="same", strides=(1,1))
                l_9_f=tf.keras.layers.Conv2D(kernel_size=(3,3), kernel_initializer="he_normal", padding="same", strides=(1,1), filters=92*n, activation=tf.keras.layers.LeakyReLU(alpha=0.3))
                l_10_f=tf.keras.layers.Conv2D(kernel_size=(5,5), kernel_initializer="he_normal", padding="same", strides=(1,1), filters=64*n, activation=tf.keras.layers.LeakyReLU(alpha=0.3))
                l_11_f=tf.keras.layers.Conv2D(kernel_size=(1,1), kernel_initializer="he_normal", activation=tf.keras.layers.LeakyReLU(alpha=0.3), strides=(1,1), filters=32*n)
            else:
                l_4=tf.keras.layers.Conv2D(kernel_size=(1,1), filters=32*n, kernel_initializer="he_normal", activation=tf.keras.layers.LeakyReLU(alpha=0.3), strides=(1,1))
                l_5=tf.keras.layers.Conv2D(kernel_size=(1,1), filters=32*n, kernel_initializer="he_normal", activation=tf.keras.layers.LeakyReLU(alpha=0.3),strides=(1,1))
                l_6=tf.keras.layers.Conv2D(kernel_size=(1,1), filters=32*n, kernel_initializer="he_normal", activation=tf.keras.layers.LeakyReLU(alpha=0.3), strides=(1,1))
                l_7_f=tf.keras.layers.MaxPooling2D(pool_size=(3,3), padding="same", strides=(2,2))
                l_9_f=tf.keras.layers.Conv2D(kernel_size=(3,3), padding="same", kernel_initializer="he_normal", strides=(2,2), filters=92*n, activation=tf.keras.layers.LeakyReLU(alpha=0.3))
                l_10_f=tf.keras.layers.Conv2D(kernel_size=(5,5), padding="same", kernel_initializer="he_normal", strides=(2,2), filters=64*n, activation=tf.keras.layers.LeakyReLU(alpha=0.3))
                l_11_f=tf.keras.layers.Conv2D(kernel_size=(1,1), kernel_initializer="he_normal", activation=tf.keras.layers.LeakyReLU(alpha=0.3), strides=(2,2), filters=32*n)
        l.append(l_4)
        l.append(l_5)
        l.append(l_6)
        l.append(l_7_f)
        l.append(l_9_f)
        l.append(l_10_f)
        l.append(l_11_f)
        l_incept.append(l)
        if n%2==0:
            c=c+1
            if c==k:
                n=n+1
                c=0
        else:
            n=n+1
    return l_incept
  
  def make_inception(self, ind, inp, training):
    l_4=self.l_inception[ind][0](inp)
    l_5=self.l_inception[ind][1](inp)
    l_6=self.l_inception[ind][3](inp)
    l_7_f=self.l_inception[ind][2](l_6)
    l_9_f=self.l_inception[ind][4](l_4)
    l_9_f_1=self.l_inception[ind][5](l_9_f)
    l_10_f=self.l_inception[ind][6](l_5)
    l_10_f_1=self.l_inception[ind][7](l_10_f)
    l_11=tf.keras.layers.concatenate([l_7_f, l_9_f_1, l_10_f_1], axis=3)
    return l_11
  def make_inception_two(self, ind, inp, training):
    l_4=self.l_inception[ind][0](inp)
    l_5=self.l_inception[ind][1](inp)
    l_6=self.l_inception[ind][3](inp)
    l_7_f=self.l_inception[ind][2](l_6)
    l_9_f=self.l_inception[ind][4](l_4)
    l_10_f=self.l_inception[ind][5](l_5)
    l_11_f=self.l_inception[ind][6](inp)
    l_11=tf.keras.layers.concatenate([l_7_f, l_9_f, l_10_f, l_11_f], axis=3)
    return l_11
  def call(self, inputs, training=False):
    l_1=self.l_1(inputs)
    l_2=self.l_2(l_1)
    l_2_1=self.l_2_1(l_2, training=training)
    l_3=self.make_inception_two(0, l_2_1, training=training)
    l_4_1=self.l_4_1(l_3, training=training)
    l_4=self.l_4(l_4_1)
    l_5=self.make_inception_two(1, l_4, training=training)
    l_7_1=self.make_inception_two(2, l_5, training=training)
    l_7_2=self.make_inception_two(3, l_7_1, training=training)
    l_7=self.make_inception_two(4, l_7_2, training=training)
    l_8=self.make_inception_two(5, l_7, training=training)
    l_9=self.make_inception_two(6, l_8, training=training)
    l_10=self.make_inception_two(7, l_9, training=training)
    l_11=self.l_10(l_10)
    l_14=self.l_13(l_11)
    #l_15=self.l_13_1(l_14)
    l_15=self.l_14(l_14)
    l_f=self.l_15(l_15)
    return l_f

  @tf.function
  def train_step(self, data):
    x= data
    with tf.GradientTape() as tape:
      y_pred = self(x, training=False)
      #y_pred = tf.divide(y_pred, tf.linalg.norm(y_pred, ord=2, keepdims=False))
      #y_pred=tf.math.l2_normalize(y_pred)
      anchor = self(self.anchor, training=False)
      #anchor=tf.math.l2_normalize(anchor)
      #anchor = tf.divide(anchor, tf.linalg.norm(anchor, ord=2, keepdims=False))
      positive = self(self.positive[0, :, :, :, :], training=False)
      #positive=tf.math.l2_normalize(positive)
      #positive = tf.divide(positive, tf.linalg.norm(positive, ord=2, keepdims=False))
      loss = CustomLoss(anchor=anchor, positive=positive, y_pred=y_pred).call(y_pred=y_pred)
        # Compute gradients
    trainable_vars = self.trainable_variables
    gradients = tape.gradient(loss, trainable_vars)

        # Update weights
    self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        # Compute our own metrics
    loss_tracker.update_state(loss)
    return {"loss": loss_tracker.result()}

  @property
  def metrics(self):
        # We list our `Metric` objects here so that `reset_states()` can be
        # called automatically at the start of each epoch
        # or at the start of `evaluate()`.
        # If you don't implement this property, you have to call
        # `reset_states()` yourself at the time of your choosing.
    return [loss_tracker]

  

In [62]:

print("completed")
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  print(tf.config.experimental.get_memory_usage('GPU:0'))
op=tf.keras.optimizers.SGD(
    learning_rate=0.00001)
#anchor=da.from_array(anchor)
#positive=da.from_array(positive)
model=CustomModel(anchor=anchor.reshape(1,160, 160, 3), positive=l_positive)
model.compile(optimizer=op)
model.fit(epochs=50, x=ims, batch_size=64, verbose = 1)


completed
106570240
Epoch 1/50
(None, 128)
(None, 128)
3166/3166 [==============================] - 1618s 508ms/step - loss: 3.0632
Epoch 2/50
3166/3166 [==============================] - 1612s 509ms/step - loss: 0.1765
Epoch 3/50
3166/3166 [==============================] - 1754s 554ms/step - loss: 0.0782
Epoch 4/50
  64/3166 [..............................] - ETA: 28:41 - loss: 0.0323

KeyboardInterrupt: 

In [74]:
## ph_comp=cv2.imread("./positive_anchor/photo_compare.jpg")
import sklearn
def euc_dis(p1, p2):
    sum_sq = np.sum(np.square(p1-p2))
    return np.sqrt(sum_sq)
im=face_detector("./positive_anchor/test_2.jpg")
if im is not None:
    f_d=convert(im, model)
    dist=euc_dis(f_d,convert(l_positive[0, :, :, :, :], model))
    dist=dist*6
    print(dist)
vid = cv2.VideoCapture(0)
while(True):
      
    # Capture the video frame
    # by frame
    ret, frame = vid.read()
    # Display the resulting frame
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 4)
    for (x, y, w, h) in faces:
        photo=frame[y:y+h,x:x+w, :]
        
        cv2.imshow('frame', photo)
        
        im_=cv2.resize(photo, (160, 160))
        im_=im_.astype(np.float32)
        im_=(im_-np.mean(im_))/np.std(im_)
        f_d=convert(im_, model)
        pos=convert(l_positive[0, :, :, :, :], model)
        dist=euc_dis(f_d, pos)
        dist=dist*6
        print(dist)
        if dist>0.6:
            print("unable")
        else:
            print("success")

    # the 'q' button is set as the
    # quitting button you may use any
    # desired button of your choice
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

0.9317637383937836
0.5190076678991318
success
0.5203875750303268
success
0.7376421242952347
unable
0.8453105092048645
unable
0.8434496819972992
unable
0.8748961687088013
unable
0.8350441753864288
unable
0.7839521169662476
unable
0.7839521169662476
unable
0.8330322504043579
unable
0.8330322504043579
unable
0.867167204618454
unable
0.8434445858001709
unable
0.9285751283168793
unable
0.9362369477748871
unable
0.824607789516449
unable
0.6549215465784073
unable
0.7135917395353317
unable
0.8269849419593811
unable
0.6825893819332123
unable
0.8747428357601166
unable
0.649031326174736
unable
0.6602391600608826
unable
0.6996773779392242
unable
0.6604379564523697
unable
0.6524176597595215
unable
0.6069089770317078
unable
0.5870680510997772
success
0.6030549556016922
unable
0.6401406079530716
unable
0.8755980134010315
unable
0.606020450592041
unable
0.6097663789987564
unable
0.5951078832149506
success
0.5951078832149506
success
0.5990273058414459
success
0.6169010996818542
unable
0.587605655193328

KeyboardInterrupt: 

In [73]:

model.save_weights('weights')
print("done")